In [43]:
import math
import random
import time
from IPython.display import clear_output

In [2]:
def defensive_heuristic1(color, bl, wl):
    if color is -1:
        return 2*len(wl) + random.random()
    else:
        return 2*len(bl) + random.random()

In [3]:
def offensive_heuristic1(color, bl, wl):
    if color is -1:
        return 2*(30-len(bl)) + random.random()
    else:
        return 2*(30-len(wl)) + random.random()

In [4]:
def defensive_heuristic2(color, bl, wl):
    val = 0
    if color is -1:   # white
        for b in bl:
            val +=  9 - b[0]   
        return val + 2*len(wl) + random.random()
    else:             # black
        for w in wl:
            val +=  w[0]   
        return val + 2*len(bl) + random.random()

In [5]:
def offensive_heuristic2(color, bl, wl):
    val = 0
    if color is -1:   # white
        for w in wl:
            val +=  9 - w[0]
        return val + 2*(30-len(bl)) + random.random()
    else:             # black
        for b in bl:
            val +=  b[0]
        return val + 2*(30-len(wl)) + random.random()

In [6]:
# 5 X 10
def init_pieces():
    wl = set()
    bl = set()
    for i in range(5):
        bl.add((0, i))
        bl.add((1, i))
        wl.add((8, i))
        wl.add((9, i))
    return bl, wl

In [7]:
bl, wl = init_pieces()
node_expanded = 0

In [8]:
# returns boolean, and the piece eaten
def viable(color, new_pos, straight, bl, wl):
    if color == 1:  # black moving
        if new_pos[0] > 0 and new_pos[1] > 0 and new_pos[0] < 10 and new_pos[1] < 5:
            # check whether moving onto black piece
            if new_pos in bl:
                return False, False
            
            # check whether moving onto white piece
            if straight:
                if new_pos in wl:
                    return False, False
            else: # may eat some piece
                if new_pos in wl:
                    return True, True, new_pos
                
            return True, False
        
        else:
            return False, False
    
    else:           # white moving
        if new_pos[0] > 0 and new_pos[1] > 0 and new_pos[0] < 10 and new_pos[1] < 5:
            # check whether moving onto white piece
            if new_pos in wl:
                return False, False
            
            # check whether moving onto black piece
            if straight:
                if new_pos in bl:
                    return False, False
            else: # may eat some piece
                if new_pos in bl:
                    return True, True, new_pos
                
            return True, False
        
        else:
            return False, False

In [15]:
def base_case(bl, wl):
    if (not wl) or (not bl):
        return True
    
    for p in bl:
        if p[0] == 9:
            return True
    for p in wl:
        if p[0] == 0:
            return True
    
    return False

In [40]:
# black --> 1
# white --> -1

def maxmove(color, bl, wl, depth, init_color, a, b):
    
    # base case: all pieces of one agent die   or   one pieces of a agent reach the other side
    if base_case(bl, wl) or depth == 0:
        if (init_color == 1):   # heuristic for black
            return offensive_heuristic2(init_color, bl, wl), -1    # the second thing return is a dummy
        else:                   # heursitc for white
            return defensive_heuristic1(init_color, bl, wl), -1    # the second thing return is a dummy 
    
    global node_expanded
    global node_bywhite
    global node_byblack
    node_expanded += 1
    if init_color == 1:  # if black inited this search
        node_byblack += 1
    else:                # if white inited this search
        node_bywhite += 1
    
    
    v = -math.inf
    if color == 1:
        cur_max = -math.inf
        best_action = (-1, bl, wl)
        # also record best action
        for p in bl:
            
            # left forward
            new_pos = (p[0]+1, p[1]+1)
            via = viable(1, new_pos, False, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                if via[1]:
                    new_wl.remove(via[2])
                    
                val = minmove(-1, new_bl, new_wl, depth-1, init_color, a, b)
                v = max(v, val[0])
                if (val[0] > cur_max):
                    cur_max = val[0]
                    best_action = (-1, new_bl, new_wl)
                if v >= b:
                    cur_max = v
                    break
                a = max(a, v)
            
            # right forward
            new_pos = (p[0]+1, p[1]-1)        
            via = viable(1, new_pos, False, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                if via[1]:
                    new_wl.remove(via[2])
                    
                val = minmove(-1, new_bl, new_wl, depth-1, init_color, a, b)
                v = max(v, val[0])                
                if (val[0] > cur_max):
                    cur_max = val[0]
                    best_action = (-1, new_bl, new_wl)
                if v >= b:
                    cur_max = v
                    break
                a = max(a, v)
                
                
            # forward
            new_pos = (p[0]+1, p[1])
            via = viable(1, new_pos, True, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                val = minmove(-1, new_bl, new_wl, depth-1, init_color, a, b)
                v = max(v, val[0])
                if (val[0] > cur_max):
                    cur_max = val[0]
                    best_action = (-1, new_bl, new_wl)
                if v >= b:
                    cur_max = v
                    break
                a = max(a, v)
                
                
            
           
        # return the most promising result after loop through all pieces
        return cur_max, best_action
    else:  # color = -1
        cur_max = -math.inf
        best_action = (1, bl, wl)
        # also record best action
        for p in wl:
            
            # left forward
            new_pos = (p[0]-1, p[1]-1)
            via = viable(-1, new_pos, False, bl, wl)
            if via[0]:
                new_wl = wl.copy()
                new_wl.remove(p)
                new_wl.add(new_pos)
                new_bl = bl.copy()
                if via[1]:
                    new_bl.remove(via[2])
                    
                val = minmove(1, new_bl, new_wl, depth-1, init_color, a, b)
                v = max(v, val[0])
                if (val[0] > cur_max):
                    cur_max = val[0]
                    best_action = (1, new_bl, new_wl)
                if v >= b:
                    cur_max = v
                    break
                a = max(a, v)
            
            
            # right forward
            new_pos = (p[0]-1, p[1]+1)        
            via = viable(-1, new_pos, False, bl, wl)
            if via[0]:
                new_wl = wl.copy()
                new_wl.remove(p)
                new_wl.add(new_pos)
                new_bl = bl.copy()
                if via[1]:
                    new_bl.remove(via[2])
                    
                val = minmove(1, new_bl, new_wl, depth-1, init_color, a, b)
                v = max(v, val[0])
                if (val[0] > cur_max):
                    cur_max = val[0]
                    best_action = (1, new_bl, new_wl)
                if v >= b:
                    cur_max = v
                    break
                a = max(a, v)
                
            
            # forward
            new_pos = (p[0]-1, p[1])
            via = viable(-1, new_pos, True, bl, wl)
            if via[0]:
                new_wl = wl.copy()
                new_wl.remove(p)
                new_wl.add(new_pos)
                new_bl = bl.copy()
                val = minmove(1, new_bl, new_wl, depth-1, init_color, a, b)
                v = max(v, val[0])
                if (val[0] > cur_max):
                    cur_max = val[0]
                    best_action = (1, new_bl, new_wl)
                if v >= b:
                    cur_max = v
                    break
                a = max(a, v)
                
                
           
        # return the most promising result after loop through all pieces
        return cur_max, best_action

In [41]:
# black --> 1
# white --> -1

def minmove(color, bl, wl, depth, init_color, a, b):
    
    # base case: all pieces of one agent die   or   one pieces of a agent reach the other side
    if base_case(bl, wl) or depth == 0:
        if (init_color == 1):   # heuristic for black
            return offensive_heuristic2(init_color, bl, wl), -1    # the second thing return is a dummy
        else:                   # heursitc for white
            return defensive_heuristic1(init_color, bl, wl), -1    # the second thing return is a dummy 

    global node_expanded
    global node_bywhite
    global node_byblack
    node_expanded += 1
    if init_color == 1:  # if black inited this search
        node_byblack += 1
    else:                # if white inited this search
        node_bywhite += 1
    
    v = math.inf
    
    if color == 1:
        cur_min = math.inf
        best_action = (-1, bl, wl)
        # also record best action
        for p in bl:
            
            # left forward
            new_pos = (p[0]+1, p[1]+1)
            via = viable(1, new_pos, False, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                if via[1]:
                    new_wl.remove(via[2])
                    
                val = maxmove(-1, new_bl, new_wl, depth-1, init_color, a, b)
                v = min(v, val[0])
                if (val[0] < cur_min):
                    cur_min = val[0]
                    best_action = (-1, new_bl, new_wl)
                if (v <= a):
                    cur_min = v
                    break
                b = min(b, v)
                       
                    
            # right forward
            new_pos = (p[0]+1, p[1]-1)        
            via = viable(1, new_pos, False, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                if via[1]:
                    new_wl.remove(via[2])
                    
                val = maxmove(-1, new_bl, new_wl, depth-1, init_color, a, b)
                v = min(v, val[0])
                if (val[0] < cur_min):
                    cur_min = val[0]
                    best_action = (-1, new_bl, new_wl)
                if (v <= a):
                    cur_min = v
                    break
                b = min(b, v)
                
                
                
            # forward
            new_pos = (p[0]+1, p[1])
            via = viable(1, new_pos, True, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                val = maxmove(-1, new_bl, new_wl, depth-1, init_color, a, b)
                v = min(v, val[0])
                if (val[0] < cur_min):
                    cur_min = val[0]
                    best_action = (-1, new_bl, new_wl)
                if (v <= a):
                    cur_min = v
                    break
                b = min(b, v)
                
                
             
           
        # return the most promising result after loop through all pieces
        return cur_min, best_action
    else:  # color = -1
        cur_min = math.inf
        best_action = (-1, bl, wl)
        # also record best action
        for p in wl:
            
            # left forward
            new_pos = (p[0]-1, p[1]-1)
            via = viable(-1, new_pos, False, bl, wl)
            if via[0]:
                new_wl = wl.copy()
                new_wl.remove(p)
                new_wl.add(new_pos)
                new_bl = bl.copy()
                if via[1]:
                    new_bl.remove(via[2])
                    
                val = maxmove(1, new_bl, new_wl, depth-1, init_color, a, b)
                v = min(v, val[0])
                if (val[0] < cur_min):
                    cur_min = val[0]
                    best_action = (1, new_bl, new_wl)
                if (v <= a):
                    cur_min = v
                    break
                b = min(b, v)
            
            
            # right forward
            new_pos = (p[0]-1, p[1]+1)        
            via = viable(-1, new_pos, False, bl, wl)
            if via[0]:
                new_wl = wl.copy()
                new_wl.remove(p)
                new_wl.add(new_pos)
                new_bl = bl.copy()
                if via[1]:
                    new_bl.remove(via[2])
                    
                val = maxmove(1, new_bl, new_wl, depth-1, init_color, a, b)
                v = min(v, val[0])
                if (val[0] < cur_min):
                    cur_min = val[0]
                    best_action = (1, new_bl, new_wl)
                if (v <= a):
                    cur_min = v
                    break
                b = min(b, v)
                
            
            # forward
            new_pos = (p[0]-1, p[1])
            via = viable(-1, new_pos, True, bl, wl)
            if via[0]:
                new_wl = wl.copy()
                new_wl.remove(p)
                new_wl.add(new_pos)
                new_bl = bl.copy()
                val = maxmove(1, new_bl, new_wl, depth-1, init_color, a, b)
                v = min(v, val[0])
                if (val[0] < cur_min):
                    cur_min = val[0]
                    best_action = (1, new_bl, new_wl)
                if (v <= a):
                    cur_min = v
                    break
                b = min(b, v)
                
                
           
        # return the most promising result after loop through all pieces
        return cur_min, best_action

In [50]:
bl, wl = init_pieces()
color = 1
counter = 0
node_expanded = 0
node_bywhite = 0
node_byblack = 0
total_time = 0
while not base_case(bl, wl):
    start_time = time.time()
    _, bl, wl = maxmove(color, bl, wl, 5, color, -math.inf, math.inf)[1]
    total_time += (time.time() - start_time)
    print(counter)

    color = -color
    counter += 1
    
if (not wl):
    print("Black wins")
elif (not bl):
    print("White wins")
else:    
    
    for p in wl:
        if p[0] == 0:
            print("White wins")
    for p in bl:
        if p[0] == 9:
            print("Black wins")
        
print("Total nodes expanded", node_expanded)
print("node expanded by white player is", node_bywhite, "node expanded by black player is", node_byblack)
print("average nodes expanded by each player is", node_expanded / counter)
print("time cost per move", total_time / counter)
board = [[0 for x in range(5)] for y in range(10)]
update_board(bl, wl)
print_board()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
Black wins
Total nodes expanded 573515
node expanded by white player is 285317 node expanded by black player is 288198
average nodes expanded by each player is 6037.0
time cost per move 0.2792838498165733
---------------------
| 0 | 0 | 0 | 0 | 0 |
---------------------
| 0 | 0 | 0 | w | w |
---------------------
| 0 | 0 | b | b | 0 |
---------------------
| 0 | 0 | 0 | 0 | 0 |
---------------------
| 0 | w | w | w | w |
---------------------
| 0 | 0 | w | 0 | w |
---------------------
| 0 | 0 | 0 | w | 0 |
---------------------
| 0 | 0 | 0 | b | 0 |
---------------------
| 0 | b | 0 | b | b |
---------------------
| 0 | 0 | 0 | b | 0 |
---------------------


In [51]:
blackwins = 0
whitewins = 0
for i in range(25):
    bl, wl = init_pieces()
    color = 1
    counter = 0
    node_expanded = 0
    while not base_case(bl, wl):
        _, bl, wl = maxmove(color, bl, wl, 4, color, -math.inf, math.inf)[1]
        color = -color
        counter += 1
    if not wl:
        print("Black wins")
        blackwins += 1
    elif not bl:
        print("White wins")
        whitewins += 1
    else:  

        for p in bl:
            if p[0] == 9:
                print("Black wins")
                blackwins += 1
                break

        for p in wl:
            if p[0] == 0:
                print("White wins")
                whitewins += 1
                break
                
    print("blackwins", blackwins, "whitewins", whitewins)

Black wins
blackwins 1 whitewins 0
Black wins
blackwins 2 whitewins 0
Black wins
blackwins 3 whitewins 0
Black wins
blackwins 4 whitewins 0
Black wins
blackwins 5 whitewins 0
Black wins
blackwins 6 whitewins 0
Black wins
blackwins 7 whitewins 0
Black wins
blackwins 8 whitewins 0
Black wins
blackwins 9 whitewins 0
Black wins
blackwins 10 whitewins 0
Black wins
blackwins 11 whitewins 0
Black wins
blackwins 12 whitewins 0
Black wins
blackwins 13 whitewins 0
Black wins
blackwins 14 whitewins 0
Black wins
blackwins 15 whitewins 0
Black wins
blackwins 16 whitewins 0
Black wins
blackwins 17 whitewins 0
Black wins
blackwins 18 whitewins 0
Black wins
blackwins 19 whitewins 0
Black wins
blackwins 20 whitewins 0
Black wins
blackwins 21 whitewins 0
Black wins
blackwins 22 whitewins 0
Black wins
blackwins 23 whitewins 0
Black wins
blackwins 24 whitewins 0
Black wins
blackwins 25 whitewins 0


In [18]:
# return a tuple:       1.  whether the move is valid
#                       2.  whether the move is able to eat a black piece
def human_viable(old_row, old_col, new_row, new_col, wl, bl):
    if (new_row, new_col) in wl:
        return False, 0
    if (new_row < 0) or (new_row > 9) or (new_col < 0) or (new_col > 4):
        return False, 0
    
    if new_row == (old_row - 1):
        if (new_col == (old_col+1)) or (new_col == (old_col-1)):
            return True, 1
        if new_col == old_col:
            if (new_row, new_col) not in bl:
                return True, 0
            else:
                return False, 0
    return False, 0

In [19]:
def update_board(bl, wl):
    
    for b in bl:
        board[b[0]][b[1]] = "b"
    for w in wl:
        board[w[0]][w[1]] = "w"

In [20]:
def print_board():
    print("---------------------")
    for i in range(10):
        print("|", board[i][0], "|", board[i][1], "|", board[i][2], "|", board[i][3], "|", board[i][4], "|")
        print("---------------------")

In [ ]:
bl, wl = init_pieces()
#print(bl)
#print(wl)
board = [[0 for x in range(5)] for y in range(10)]
update_board(bl, wl)
print_board()
while(True):
    
    clear_output(wait=True)
    
    # receive coord of the piece that user wanna move
    selected_x = int(input("Please enter x coord of the piece that you wanna move: "))
    selected_y = int(input("Please enter y coord of the piece that you wanna move: "))
    
    # receive coord of position that user wanna move the selected piece to
    target_x = int(input("Please enter x coord that you wanna move to: "))
    target_y = int(input("Please enter y coord that you wanna move to: "))

    # check if that is viable and update bl, wl
    hv = human_viable(selected_y, selected_x, target_y, target_x, wl, bl)
    if hv[0]:
        wl.remove((selected_y, selected_x))
        wl.add((target_y, target_x))
        if hv[1]:      # if the move white made is able to eat a black piece
            bl.discard((target_y, target_x))
            
        _, bl, wl = maxmove(1, bl, wl, 3, 1, -math.inf, math.inf)[1]
    else:
        print("Invalid Move!!!!")
        
    board = [[0 for x in range(5)] for y in range(10)]
    update_board(bl, wl)
    print_board()

Please enter x coord of the piece that you wanna move: 2
Please enter y coord of the piece that you wanna move: 8
Please enter x coord that you wanna move to: 2
Please enter y coord that you wanna move to: 7
---------------------
| b | b | b | b | b |
---------------------
| b | b | b | b | 0 |
---------------------
| 0 | 0 | 0 | b | 0 |
---------------------
| 0 | 0 | 0 | 0 | 0 |
---------------------
| 0 | 0 | 0 | 0 | 0 |
---------------------
| 0 | 0 | 0 | 0 | 0 |
---------------------
| 0 | 0 | 0 | 0 | 0 |
---------------------
| 0 | 0 | w | 0 | 0 |
---------------------
| w | w | 0 | w | w |
---------------------
| w | w | w | w | w |
---------------------
